In [ ]:
# Importing important libraries and modules
import cv2
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Dropout, Conv2D, Dense, BatchNormalization, AveragePooling2D, MaxPooling2D, Flatten
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from zipfile import ZipFile

In [ ]:
# Extracting the training data and test data for prediction from the zip file

# for training data
with ZipFile('../input/dogs-vs-cats/train.zip', 'r') as zip: 
    print('Extracting all the files for training data now...') 
    zip.extractall() 
    print('Done!')
    
# for test data for prediction   
with ZipFile('../input/dogs-vs-cats/test1.zip', 'r') as zip: 
    print('Extracting all the files for test data for prediction now...') 
    zip.extractall() 
    print('Done!')    
            

In [ ]:
# We have to read the training data from a file which contains data in the form of image. 
# The folder is named as 'train' and it contains images of dogs and cats

# First of all we will extract the detail of all the data and save all of them in terms of dataframe with foldername, imagename, objectname and labels
detail = glob.glob("./train/*")
foldername = [str(i.split("n/")[0]) + "n" for i in detail]
imagename = [str(i.split("/")[2]) for i in detail]
objectname = [str(i.split(".")[0]) for i in imagename]
label = [1 if i == 'dog' else 0 for i in objectname]

# Defining dataframe and saving all the extracted information in that dataframe
train_data_detail = pd.DataFrame() 
train_data_detail["foldername"] = foldername
train_data_detail["imagename"] = imagename
train_data_detail["objectname"] = objectname
train_data_detail["label"] = label

# Shuffling of all the classes present in that dataframe
train_data_detail.reset_index(inplace = True, drop = True)  

# Analying the train data detail
print("\nNumber of images in training set = "+str(len(detail)))
print(train_data_detail.columns)
train_data_detail.head()

In [ ]:
# Splitting training set into initial training set and test set
train_data_detail, test_data_detail = train_test_split(train_data_detail, stratify=train_data_detail["label"], test_size = 0.08)

# Splitting training data into final training set and cross validation set
train_data_detail, cv_data_detail = train_test_split(train_data_detail, stratify=train_data_detail["label"], test_size = 0.086956)
train_data_detail.shape, test_data_detail.shape, cv_data_detail.shape

In [ ]:
# Resetting index of train, cross validation and test set
train_data_detail.reset_index(inplace = True, drop = True)
cv_data_detail.reset_index(inplace = True, drop = True)
test_data_detail.reset_index(inplace = True, drop = True)

In [ ]:
# plotting  and printing distribution of each class in all train, cross validation and test set

# for training data.................................................................................................................
fig = plt.figure(figsize = (10, 6))
ax = fig.add_axes([0,0,1,1])
ax.set_title("object in Training Data set", fontsize = 20)
sns.countplot(x = "objectname", data = train_data_detail)
for i in ax.patches:
    ax.text(x = i.get_x() + 0.2, y = i.get_height()+1.5, s = str(i.get_height()), fontsize = 20, color = "black")
plt.xlabel("")
plt.ylabel("Count", fontsize = 13)
plt.tick_params(labelsize = 15)
plt.xticks(rotation = 0)
plt.show()

# for cross validation data............................................................................................................
fig = plt.figure(figsize = (10, 6))
ax = fig.add_axes([0,0,1,1])
ax.set_title("object in cv Data set", fontsize = 20)
sns.countplot(x = "objectname", data = cv_data_detail)
for i in ax.patches:
    ax.text(x = i.get_x() + 0.2, y = i.get_height()+1.5, s = str(i.get_height()), fontsize = 20, color = "black")
plt.xlabel("")
plt.ylabel("Count", fontsize = 13)
plt.tick_params(labelsize = 15)
plt.xticks(rotation = 0)
plt.show()

# for test data............................................................................................................................
fig = plt.figure(figsize = (10, 6))
ax = fig.add_axes([0,0,1,1])
ax.set_title("object in test Data set", fontsize = 20)
sns.countplot(x = "objectname", data = test_data_detail)
for i in ax.patches:
    ax.text(x = i.get_x() + 0.2, y = i.get_height()+1.5, s = str(i.get_height()), fontsize = 20, color = "black")
plt.xlabel("")
plt.ylabel("Count", fontsize = 13)
plt.tick_params(labelsize = 15)
plt.xticks(rotation = 0)
plt.show()

In [ ]:
# Changing the data into an array of pixels and labels so that it can be fed into the model expect test which is for prediction only
# Initially it was in the form of a DataFrame

# for training data
train_x = []
train_y = []
for i in range(len(train_data_detail)):
        path1 = train_data_detail["foldername"][i]
        path2 = train_data_detail["imagename"][i]
        image = cv2.imread(os.path.join(path1, path2))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (128, 128))
        #here, we are normalizing the images
        norm_image = image/255.0 
        #Creating and saving each image in the form of numerical data in an array 
        train_x.append(norm_image)
        #appending corresponding labels 
        train_y.append(train_data_detail['label'][i])  
train_x = np.array(train_x)
train_y = np.array(train_y)
print(" for training data ", train_x.shape, train_y.shape)

# for test data
cv_x = []
cv_y = []
for i in range(len(cv_data_detail)):
        path1 = cv_data_detail["foldername"][i]
        path2 = cv_data_detail["imagename"][i]
        image = cv2.imread(os.path.join(path1, path2))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (128, 128))
        #here, we are normalizing the images
        norm_image = image/255.0 
        #Creating and saving each image in the form of numerical data in an array 
        cv_x.append(norm_image)
        #appending corresponding labels 
        cv_y.append(cv_data_detail['label'][i])  
cv_x = np.array(cv_x)
cv_y = np.array(cv_y)
print(" for cv data ",cv_x.shape, cv_y.shape)

# for cv data
test_x = []
test_y = []
for i in range(len(test_data_detail)):
        path1 = test_data_detail["foldername"][i]
        path2 = test_data_detail["imagename"][i]
        image = cv2.imread(os.path.join(path1, path2))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (128, 128))
        #here, we are normalizing the images
        norm_image = image/255.0 
        #Creating and saving each image in the form of numerical data in an array 
        test_x.append(norm_image)
        #appending corresponding labels 
        test_y.append(test_data_detail['label'][i])  
test_x = np.array(test_x)
test_y = np.array(test_y)
print(" for test data ",test_x.shape, test_y.shape)

In [ ]:
# Checking the accuracy after processing
r = 10000
plt.imshow(train_x[r])
print(train_y[r])

In [ ]:
# We also need to read the test data for prediction from a file which contains data in the form of image. 
# The folder is named as 'test1' and it contains images of dogs and cats

# First of all we will extract the detail of all the data and save all of them in terms of dataframe with foldername and imagename only
detail = glob.glob("./test1/*")
foldername = [str(i.split("1/")[0]) + "1" for i in detail]
imagename = [str(i.split("/")[2]) for i in detail]

# Defining dataframe and saving all the extracted information in that dataframe
test_data_for_prediction_detail = pd.DataFrame() 
test_data_for_prediction_detail["foldername"] = foldername
test_data_for_prediction_detail["imagename"] = imagename

# Analying the test data set for prediction detail
print("\nNumber of images in test data set for prediction  = "+str(len(detail)))
print(test_data_for_prediction_detail.columns)
test_data_for_prediction_detail.head()

In [ ]:
# Changing the data into an array of pixels and labels so that it can be fed into the model for prediction 
# Initially it was in the form of a DataFrame

# for test data for prediction data
prediction = []
for i in range(len(test_data_for_prediction_detail)):
        path1 = test_data_for_prediction_detail["foldername"][i]
        path2 = test_data_for_prediction_detail["imagename"][i]
        image = cv2.imread(os.path.join(path1, path2))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (128, 128))
        #here, we are normalizing the images
        norm_image = image/255.0 
        #Creating and saving each image in the form of numerical data in an array 
        prediction.append(norm_image) 
prediction = np.array(prediction)
print(" for test data for prediction ", prediction.shape)

In [ ]:
# To train and predict on the given data we need to covert them into tensor
train_x = train_x.reshape(21000, 128, 128, 1)
test_x = test_x.reshape(2000, 128, 128, 1)
cv_x = cv_x.reshape(2000, 128, 128, 1)
prediction = prediction.reshape(12500, 128, 128, 1)

In [ ]:
# Defining a model consisting of convolutional layers, polling layers and fully connected layer
def model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(128,128,1)))
    model.add(MaxPooling2D())

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dropout(0.7))

    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    return model
model = model()
model.summary()    

In [ ]:
# Compiling and running the model
model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ["accuracy"])
hist = model.fit(train_x, train_y, validation_data=(cv_x, cv_y), epochs = 50)

In [ ]:
# visualizing losses and accuracy with epochs 
epoch_number = []
for epoch in range(50):
    epoch_number.append(epoch + 1)
train_loss = hist.history['loss']
val_loss   = hist.history['val_loss']
train_acc  = hist.history['accuracy']
val_acc    = hist.history['val_accuracy']

In [ ]:
# printing a table depicting the detail about the trained model
log_frame = pd.DataFrame(columns = ["Epoch", "Train_Loss", "Train_Accuracy", "CV_Loss", "CV_Accuracy"])
log_frame["Epoch"] = epoch_number
log_frame["Train_Loss"] = train_loss
log_frame["Train_Accuracy"] = train_acc
log_frame["CV_Loss"] = val_loss
log_frame["CV_Accuracy"] = val_acc 
log_frame

In [ ]:
# plotting epoch vs loss
def plotting(epoch, train_loss, CV_loss, title):
    fig, axes = plt.subplots(1,1, figsize = (12, 8))
    axes.plot(epoch, train_loss, color = 'red', label = "Train")
    axes.plot(epoch, CV_loss, color = 'blue', label = "CV")
    axes.set_title(title, fontsize = 25)
    axes.set_xlabel("Epochs", fontsize = 20)
    axes.set_ylabel("Loss", fontsize = 20)
    axes.grid()
    axes.legend(fontsize = 20)

plotting(list(log_frame["Epoch"]), list(log_frame["Train_Loss"]), list(log_frame["CV_Loss"]), "EPOCH VS LOSS") 

In [ ]:
# plotting epoch vs accuracy
def plotting(epoch, train_acc, CV_acc, title):
    fig, axes = plt.subplots(1,1, figsize = (12, 8))
    axes.plot(epoch, train_acc, color = 'red', label = "Train_Accuracy")
    axes.plot(epoch, CV_acc, color = 'blue', label = "CV_Accuracy")
    axes.set_title(title, fontsize = 25)
    axes.set_xlabel("Epochs", fontsize = 20)
    axes.set_ylabel("Accuracy", fontsize = 20)
    axes.grid()
    axes.legend(fontsize = 20)

plotting(list(log_frame["Epoch"]), list(log_frame["Train_Accuracy"]), list(log_frame["CV_Accuracy"]), "EPOCH VS ACCURACY") 

In [ ]:
# predicting on test data
test_predict = model.predict(test_x)

In [ ]:
# Since the test_predict is in the form of dummy variable with the value of probability so we will convert it into categorical data
l = []
for i in range(test_predict.shape[0]):
  if test_predict[i] < 0.5:
    j = 0
  else:
    j = 1
  l.append(j)
test_predict = pd.DataFrame(l)

In [ ]:
# Accuracy on test data
acc = accuracy_score(test_y, l) * 100
acc

In [ ]:
# Now prediction on data to be predicted
prediction_predict = model.predict(prediction)

In [ ]:
# Since the prediction_predict is in the form of dummy variable with the value of probability so we will convert it into categorical data
l = []
for i in range(prediction_predict.shape[0]):
  if prediction_predict[i] < 0.5:
    j = 0
  else:
    j = 1
  l.append(j)
submission2 = pd.DataFrame(l)  
submission2 = submission2.rename(columns = {0 : 'label'})

In [ ]:
# loading sample submission file and dropping the falsely labelled column from Sample submission file and clubing it with the predicted file 
submission1 = pd.read_csv('../input/dogs-vs-cats/sampleSubmission.csv')
submission1.drop(columns = ['label'], inplace = True)
submission = pd.concat([submission1, submission2], axis = 1)

In [ ]:
# Final submission file
submission

In [ ]:
# Validation by image plotting
t = prediction.reshape(12500, 128, 128)
r = 233
plt.imshow(t[r])
if submission["label"][r] == 1:
    print("It is a dog")
else:
    print("it is a cat")

In [ ]:
# Delete unwanted output files
import shutil
shutil.rmtree('./train')
shutil.rmtree('./test1')

In [ ]:
# Saving the submission file
submission.to_csv('submission.csv', index = False)